#### Simple Gen AI APP Using Langchain

In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
# Lang Smith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [36]:
# Data Ingestion - From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [37]:
loader = WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [38]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='\n\n\n\n\n🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.\n\n')]

In [39]:
# Load Data --> Docs --> Divide our Documents into chunks documents --> text --> vectors --> Vector Embeddings- --> Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [40]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.')]

In [41]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [42]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [43]:
vectorstoredb

In [44]:
# Query From a vector store db
query = "LangSmith has two usage limits: total traces and extended"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.'

In [45]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [46]:
# Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11e8cb7c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11e95f970>, root_client=<openai.OpenAI object at 0x11e8cbcd0>, root_async_client=<openai.AsyncOpenAI object at 0x11e8cbca0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_

In [47]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "LangSmith has two usage limits: total traces and extended",
    "context": [Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

/Users/greatdaveo/CODE_BASE/ML-Course/venv/lib/python3.10/site-packages/langsmith/client.py:290: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=7d156d2c-64a8-42ef-896c-30eb48e6547f; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=1061bc08-f2b9-4b72-8dae-fe58b9870cc5; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=48c219ad-285e-4450-9999-c553b4605dab; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=05632605-4c17-4387-a252-7477de2dfc01; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=eb158be0-dbe0-4a34-9f53-d6eddeb63534


"LangSmith's usage limits comprise two elements: total traces and extended traces. These are the two metrics that are tracked on the usage graph."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [48]:
### Input ---> Retriever ---> vector store db

vectorstoredb

In [49]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [50]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1195e8c40>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | ChatOpenAI(

In [51]:
# Get the response form the LLM
response = retrieval_chain.invoke({"input": "LangSmith has two usage limits: total traces and extended"})
response['answer']

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=eb158be0-dbe0-4a34-9f53-d6eddeb63534; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=16714ecf-702a-4cd6-b146-a18226bbfe38; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=16714ecf-702a-4cd6-b146-a18226bbfe38; trace=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed,id=dadf1ca6-9fc7-456f-8a30-0d7fa4bce7ed; trace=24869061-0c2e-441c-b04a-986af1904a4d,id=24869061-0c2e-441c-b04a-986af1904a4d; trace=24869061-0c2e-441c-b04a-986af1904a4d,id=f4fd916c-2a35-426b-8a69-c2d745441e30; trace=24869061-0c2e-441c-b04a-986af1904a4d,id=408e5043-6484-4523-813c-80d6112879eb; trace=24869061-0c2e-441c-b04a-986af1904a4d,id=53eac48e-a8c8-45c5-84ee-64c62a8b09f0; trace=24869061-0c2e-441c-b04a-986af1904a4d,id

'What is the copyright year of the document provided in the context? \n\nCopyright © 2025 LangChain, Inc.'

In [52]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='a93c34d3-5c55-48be-9d30-0a2e5c2d23fc', metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.')],
 'answer': 'What is the copyright year of the document provided in the context? \n\nCopyright © 2025 LangChain, Inc.'}

In [53]:
response['context']

[Document(id='a93c34d3-5c55-48be-9d30-0a2e5c2d23fc', metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.')]